In [1]:
def read_data(path):
    f = open(path,"r")
    lines = f.readlines()
    template = lines[0].strip()
    insertions_lines = [ line.strip() for line in lines[2:] ]
    insertions = {p.split(" -> ")[0]: p.split(" -> ")[1] for p in insertions_lines}
    return (template, insertions)
    return [int(line.strip()) for line in f.readlines()]

In [2]:
demo_template, demo_insertions = read_data("demo.txt")
full_template, full_insertions = read_data("data.txt")
demo_template

'NNCB'

In [3]:
demo_insertions

{'CH': 'B',
 'HH': 'N',
 'CB': 'H',
 'NH': 'C',
 'HB': 'C',
 'HC': 'B',
 'HN': 'C',
 'NN': 'C',
 'BH': 'H',
 'NC': 'B',
 'NB': 'B',
 'BN': 'B',
 'BB': 'N',
 'BC': 'B',
 'CC': 'N',
 'CN': 'C'}

# Part 1

In [4]:
def insert(template: str, insertions: dict) -> str:
    new_template = ""
    for i in range(0, len(template)-1):
        left = template[i]
        right = template[i+1]
        inserted = insertions[left + right]
        new_template += left + inserted
    # finally insert the last character 
    new_template += template[-1]
    return new_template

In [5]:
assert insert(demo_template, demo_insertions) == "NCNBCHB"

In [6]:
def multi_insert(template: str, insertions: dict, steps: int) -> str:
    new_template = template
    for i in range(steps):
        new_template = insert(new_template, insertions)
    return new_template

In [7]:
assert multi_insert(demo_template, demo_insertions, 1) == "NCNBCHB"
assert multi_insert(demo_template, demo_insertions, 2) == "NBCCNBBBCBHCB"
assert multi_insert(demo_template, demo_insertions, 3) == "NBBBCNCCNBBNBNBBCHBHHBCHB"
assert multi_insert(demo_template, demo_insertions, 4) == "NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB"

In [8]:
def compute_score(template: str, insertions: dict, steps: int = 10):
    new_template = multi_insert(template, insertions, steps)
    characters = set([i for i in new_template])
    counts = { char: new_template.count(char) for char in characters }
    min_count = min(counts.values())
    max_count = max(counts.values())
    return max_count - min_count

In [9]:
assert compute_score(demo_template, demo_insertions) == 1588

In [10]:
compute_score(full_template, full_insertions)

2587

# Part 2

Let's get rid of the order and only consider the number of each pair

In [11]:
def contribute(key: str, dict_template: dict, val: int = 1) -> None:
    if key in dict_template:
        dict_template[key] += val
    else:
        dict_template[key] = val

In [12]:
def build_template(template: str) -> dict:
    dict_template = {}
    for i in range(0, len(template)-1):
        pair = template[i] + template[i+1]
        contribute(pair, dict_template)
    return dict_template      

In [13]:
assert build_template(demo_template) == {'NN': 1, 'NC': 1, 'CB': 1}

In [14]:
def insert2(template: dict, insertions: dict) -> str:
    new_template = {}
    for pair, val in template.items():
        left = pair[0]
        right = pair[1]
        inserted = insertions[pair]
        contribute(left+inserted, new_template, val)
        contribute(inserted+right, new_template, val)
    return new_template

In [15]:
assert insert2(build_template(demo_template), demo_insertions) == {'NC': 1, 'CN': 1, 'NB': 1, 'BC': 1, 'CH': 1, 'HB': 1}

In [16]:
def multi_insert2(template: dict, insertions: dict, steps: int) -> str:
    new_template = template
    for i in range(steps):
        new_template = insert2(new_template, insertions)
    return new_template

In [17]:
def compute_score(template: str, insertions: dict, steps: int = 10):
    new_template = multi_insert2(build_template(template), insertions, steps)
    char_count = {}
    for pair, val in new_template.items():
        # only contribute the left part of the pair to avoid double contributions
        contribute(pair[0], char_count, val)
    # last character has never benn contributed
    contribute(template[-1], char_count, 1)
    min_count = min(char_count.values())
    max_count = max(char_count.values())
    return max_count - min_count

In [18]:
assert compute_score(demo_template, demo_insertions, 10) == 1588

In [19]:
assert compute_score(demo_template, demo_insertions, 40) == 2188189693529

In [20]:
compute_score(full_template, full_insertions, 40)

3318837563123